### Project 1: TimeZone class

Давайте начнем с класса timezone. У него будет два атрибута экземпляра, `offset` и `name`.

Я собираюсь создать их как свойства только для чтения.

Смещения должны быть предоставлены как временной интервал (timedelta) часов и минут — мы разрешим указывать смещения часов и минут отдельно в **init**, но свойство offset объединит их как объект `timepan`.

In [1]:
import numbers
from datetime import timedelta


class TimeZone:

    def __init__(
        self,
        name: str,
        offset_hours: int,
        offset_minutes: int) -> None:

        if name is None or len(str(name).strip()) == 0:
            raise ValueError('Timezone name cannot be empty.')
        self._name = str(name).strip()

        if not isinstance(offset_hours, numbers.Integral):
            raise ValueError('Hour offset must be an integer.')

        if not isinstance(offset_minutes, numbers.Integral):
            raise ValueError('Minutes offset must be an integer.')

        if offset_minutes < -59 or offset_minutes > 59:
            raise ValueError('Minutes offset must between -59 and 59 (inclusive).')

        # for time delta sign of minutes will be set to sign of hours
        offset = timedelta(
            hours=offset_hours,
            minutes=offset_minutes
        )

        # offsets are technically bounded between -12:00 and 14:00
        # see: https://en.wikipedia.org/wiki/List_of_UTC_time_offsets
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and +14:00.')

        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset

    @property
    def offset(self):
        return self._offset

    @property
    def name(self):
        return self._name

    def __eq__(self, other):
        return (
            isinstance(other, TimeZone) and
            self.name == other.name and
            self._offset_hours == other._offset_hours and
            self._offset_minutes == other._offset_minutes
        )

    def __repr__(self):
        return (
            f"TimeZone(name='{self.name}', "
            f"offset_hours={self._offset_hours}, "
            f"offset_minutes={self._offset_minutes})"
        )

Давайте попробуем и убедимся, что это работает:

In [2]:
tz1 = TimeZone('ABC', -2, -15)

In [3]:
tz1.name

'ABC'

In [8]:
import datetime

dt = datetime.datetime.now(datetime.UTC)
print(dt)

2025-01-20 13:45:04.239271+00:00


In [5]:
print(dt + tz1.offset)

2019-06-02 21:12:15.937254


Как мы видим, смещение, похоже, работает (-2:15 от текущего времени)

(Нам действительно следует писать модульные тесты по мере написания кода, но я покажу вам модульные тесты в последнем разделе этого проекта, а в следующем проекте мы сможем писать код и модульные тесты параллельно)

---


### Лирическое отступление: тип numbers.Integral


В Python `numbers.Integral` — это абстрактный базовый класс, который определяет интерфейс для целочисленных типов. Он является частью модуля `numbers`, который предоставляет иерархию абстрактных базовых классов для числовых типов.

Основные аспекты `numbers.Integral`

- Целочисленные типы:
`numbers.Integral` представляет собой интерфейс для всех целочисленных типов, таких как `int` в Python 3. Он позволяет разработчикам проверять, является ли объект целым числом, не полагаясь на конкретные реализации.

- Использование:
Вы можете использовать `numbers.Integral` для проверки, является ли объект целым числом, с помощью функции `isinstance()`.

Например:

In [9]:
import numbers

x = 10
y = 3.14

print(isinstance(x, numbers.Integral))  # True, так как x - целое число
print(isinstance(y, numbers.Integral))  # False, так как y - не целое число


True
False


- Преимущества:
    - Полиморфизм: Использование numbers.Integral позволяет писать код, который может работать с любыми типами, реализующими этот интерфейс, что делает код более гибким и расширяемым.
    - Читаемость: Проверка типов с использованием абстрактных базовых классов делает код более понятным и самодокументируемым.

- Иерархия чисел:
Модуль numbers включает в себя несколько других абстрактных классов, таких как:
    - numbers.Number: базовый класс для всех чисел.
    - numbers.Rational: для рациональных чисел.
    - numbers.Real: для вещественных чисел.
    - numbers.Complex: для комплексных чисел.

#### Пример использования

Вот пример, который демонстрирует использование `numbers.Integral` в функции, которая принимает только целые числа:

In [10]:
import numbers

def process_integer(value):
    if not isinstance(value, numbers.Integral):
        raise TypeError("Ожидалось целое число")
    # Обработка целого числа
    return value * 2

print(process_integer(5))  # 10
# print(process_integer(3.5))  # Это вызовет TypeError


10


Таким образом, `numbers.Integral` предоставляет удобный способ работы с целыми числами и проверки типов в Python, что делает код более безопасным и понятным.

Основные типы, подпадающие под numbers.Integral:

- int: Это основной целочисленный тип в Python 3. Он представляет целые числа произвольной длины.
- Пользовательские классы: Вы можете создать собственные классы, которые наследуют от numbers.Integral и реализуют необходимые методы.
- bool: В Python bool является подтипом int, и, следовательно, также считается целым числом. Значения True и False в Python соответствуют 1 и 0 соответственно.

Например:

In [21]:
import numbers

class MyInteger(numbers.Integral):
    def __init__(self, value):
        self._value = int(value)

    def __int__(self):
        return self._value

    def __add__(self, other):
        return MyInteger(self._value + int(other))

    def __sub__(self, other):
        return MyInteger(self._value - int(other))

    def __mul__(self, other):
        return MyInteger(self._value * int(other))

    def __truediv__(self, other):
        return MyInteger(self._value // int(other))  # Целочисленное деление

    def __floordiv__(self, other):
        return MyInteger(self._value // int(other))

    def __mod__(self, other):
        return MyInteger(self._value % int(other))

    def __pow__(self, other):
        return MyInteger(self._value ** int(other))

    def __eq__(self, other):
        return self._value == int(other)

    def __lt__(self, other):
        return self._value < int(other)

    def __le__(self, other):
        return self._value <= int(other)

    def __neg__(self):
        return MyInteger(-self._value)

    def __abs__(self):
        return MyInteger(abs(self._value))

    def __invert__(self):
        return MyInteger(~self._value)

    def __and__(self, other):
        return MyInteger(self._value & int(other))

    def __or__(self, other):
        return MyInteger(self._value | int(other))

    def __xor__(self, other):
        return MyInteger(self._value ^ int(other))

    def __lshift__(self, other):
        return MyInteger(self._value << int(other))

    def __rshift__(self, other):
        return MyInteger(self._value >> int(other))

    def __ceil__(self):
        return MyInteger(self._value)

    def __floor__(self):
        return MyInteger(self._value)

    def __round__(self, n=0):
        return MyInteger(round(self._value, n))

    def __pos__(self):
        return MyInteger(+self._value)

    def __radd__(self, other):
        return self.__add__(other)

    def __rand__(self, other):
        return MyInteger(other & self._value)

    def __rfloordiv__(self, other):
        return MyInteger(other // self._value)

    def __rlshift__(self, other):
        return MyInteger(other << self._value)

    def __rmod__(self, other):
        return MyInteger(other % self._value)

    def __rmul__(self, other):
        return MyInteger(other * self._value)

    def __ror__(self, other):
        return MyInteger(other | self._value)

    def __rpow__(self, other):
        return MyInteger(other ** self._value)

    def __rrshift__(self, other):
        return MyInteger(other >> self._value)

    def __rtruediv__(self, other):
        return MyInteger(other // self._value)

    def __rxor__(self, other):
        return MyInteger(other ^ self._value)

    def __trunc__(self):
        return MyInteger(self._value)

    def __repr__(self):
        return f"MyInteger({self._value})"

# Пример использования
my_int = MyInteger(10)
print(my_int)  # MyInteger(10)
print(isinstance(my_int, numbers.Integral))  # True
print(my_int + 5)  # MyInteger(15)
print(my_int - 3)  # MyInteger(7)


MyInteger(10)
True
MyInteger(15)
MyInteger(7)


Хотя `numbers.Integral` может быть использован для проверки пользовательских классов, важно помнить, что стандартные типы, такие как `int`, являются основными типами, которые вы будете использовать в большинстве случаев. В большинстве приложений, работающих с целыми числами, вы будете иметь дело именно с `int` и, возможно, с `bool`.

Таким образом, в стандартной библиотеке `Python` `int` и `bool` — это основные типы, которые подпадают под numbers.Integral. Пользовательские классы могут быть созданы для расширения функциональности, если это необходимо.